In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.models import Model,load_model
import numpy as np
import tensorflow as tf

In [25]:
!unzip xray.zip

Archive:  xray.zip
replace xray/test/Covid_19/covid_19 1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: [n]o
error:  invalid response [[n]o]
replace xray/test/Covid_19/covid_19 1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: [N]one
error:  invalid response [[N]one]
replace xray/test/Covid_19/covid_19 1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [26]:
classifier=Sequential()

classifier.add(Convolution2D(32,3,3,input_shape=(256,256,3),activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))

classifier.add(Convolution2D(32,3,3,activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))

classifier.add(Convolution2D(64,3,3,activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))

classifier.add(Convolution2D(128,3,3,activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))

classifier.add(Flatten())

classifier.add(Dense(output_dim=128,activation='relu'))
classifier.add(Dense(output_dim=128,activation='relu'))

classifier.add(Dense(output_dim=3,activation='softmax'))

classifier.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(256, 256,..., activation="relu")`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
/usr/local/lib/pytho

In [27]:
from keras.preprocessing.image import ImageDataGenerator,img_to_array,load_img

train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory("/content/xray/train",target_size=(256,256),batch_size=10,class_mode='categorical')

testing_set = test_datagen.flow_from_directory("/content/xray/test",target_size=(256,256),batch_size=2,class_mode='categorical')




Found 600 images belonging to 3 classes.
Found 69 images belonging to 3 classes.


In [28]:
classifier.fit_generator(training_set,samples_per_epoch=600,nb_epoch=50,validation_data=testing_set,nb_val_samples=69)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., steps_per_epoch=60, epochs=50, validation_steps=69)`
  """Entry point for launching an IPython kernel.


Epoch 1/50
60/60 [==============================] - 29s 486ms/step - loss: 0.9203 - accuracy: 0.5700 - val_loss: 1.6709 - val_accuracy: 0.5401
Epoch 2/50
60/60 [==============================] - 25s 421ms/step - loss: 0.6123 - accuracy: 0.7533 - val_loss: 0.6993 - val_accuracy: 0.6397
Epoch 3/50
60/60 [==============================] - 26s 426ms/step - loss: 0.4771 - accuracy: 0.8383 - val_loss: 2.3597 - val_accuracy: 0.5956
Epoch 4/50
60/60 [==============================] - 24s 404ms/step - loss: 0.4072 - accuracy: 0.8383 - val_loss: 0.0523 - val_accuracy: 0.6765
Epoch 5/50
60/60 [==============================] - 25s 424ms/step - loss: 0.3540 - accuracy: 0.8733 - val_loss: 1.7393 - val_accuracy: 0.6838
Epoch 6/50
60/60 [==============================] - 24s 400ms/step - loss: 0.3678 - accuracy: 0.8550 - val_loss: 0.0195 - val_accuracy: 0.7794
Epoch 7/50
60/60 [==============================] - 24s 407ms/step - loss: 0.2896 - accuracy: 0.8900 - val_loss: 0.0023 - val_accuracy: 0.7132

In [0]:
classifier.save('covidrgb.h5')

In [30]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# 2. Save Keras Model or weights on google drive

# create on Colab directory
classifier.save('covidrgbfinal.h5')    
model_file = drive.CreateFile({'title' : 'covidrgbfinal.h5'})
model_file.SetContentFile('covidrgbfinal.h5')
model_file.Upload()

# download to google drive
drive.CreateFile({'id': model_file.get('id')})

GoogleDriveFile({'id': '1QNw8cGeTHb0WwM2l15VFFw7cA1U0nE1c'})

In [0]:

model=load_model('covidrgbfinal.h5')

In [52]:
from keras.preprocessing import image
test_image=image.load_img('/content/Pneumonia 12.jpg',target_size=(256,256))

test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)
proba = model.predict(test_image)
class_idx = np.argmax(proba)
print(proba)

if class_idx==0:
    label="Covid-19"
elif class_idx==1:
    label="Normal"
elif class_idx==2:
    label="Pneumonia"
else:
    label="Unknown"

# label = "{}: {:.2f}%".format(label, proba[class_idx] * 100)
print("Result:- "+str(label))



[[0. 0. 1.]]
Result:- Pneumonia
